In [1]:
import random
import numpy as np
import torch

In [2]:
from generation import Generation
from evolution_parameters import EvolutionParameters
from genome_parameters import GenomeParameters

In [3]:
population_cnt = 100
mutation_prob = 0.3
crossover_prob = 0.1

# Testing generations

In [4]:


generation = Generation(population_cnt, 
                 EvolutionParameters(mutation_prob, crossover_prob), 
                 GenomeParameters(max_hidden_layers=10))

TypeError: EvolutionParameters.__init__() missing 4 required positional arguments: 'training_data', 'test_data', 'loss_fn', and 'optimizer'

In [8]:
vars(generation)

{'device': 'mps',
 'population_cnt': 100,
 'evolution_params': <evolution_parameters.EvolutionParameters at 0x28ad41430>,
 'genome_params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'population': [<individual.Individual at 0x28aff1370>,
  <individual.Individual at 0x28b8d8860>]}

# Testing mutation

In [9]:
ind = generation.population[random.randint(0, generation.population_cnt -1)]
vars(ind)


{'genome': <genomes.Genome at 0x28aff2180>,
 'evolution_params': <evolution_parameters.EvolutionParameters at 0x28ad41430>,
 'genome_params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'device': 'mps'}

In [10]:
genome = ind.genome
vars(genome)

{'params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'epochs': 44,
 'batch_size': 69,
 'hidden_layers': 3,
 'activation_fns': [torch.nn.modules.linear.Linear,
  torch.nn.modules.activation.Softmax,
  torch.nn.modules.activation.Softmax],
 'neuron_cnt': array([86, 99, 99])}

In [12]:
for _ in range(5): ind.mutate()

IndexError: index 5 is out of bounds for axis 0 with size 1

In [13]:
genome = ind.genome
vars(genome)

{'params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'epochs': 33,
 'batch_size': 39,
 'hidden_layers': 6,
 'activation_fns': array([<class 'torch.nn.modules.linear.Linear'>], dtype=object),
 'neuron_cnt': array([12])}

# Testing crossover

In [3]:
from crossover import Crossover

In [16]:
inds = np.random.choice(generation.population, 2, replace=False)
ind1 = inds[0]
ind2 = inds[1]

In [17]:
vars(ind1.genome)

{'params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'epochs': 4,
 'batch_size': 68,
 'hidden_layers': 8,
 'activation_fns': [torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.Softmax,
  torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.GELU,
  torch.nn.modules.activation.PReLU,
  torch.nn.modules.activation.GELU,
  torch.nn.modules.linear.Linear],
 'neuron_cnt': array([22, 22, 22, 22, 22, 22, 22, 22])}

In [18]:
vars(ind2.genome)

{'params': <genome_parameters.GenomeParameters at 0x28aff1430>,
 'epochs': 21,
 'batch_size': 19,
 'hidden_layers': 8,
 'activation_fns': [torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.Softmin,
  torch.nn.modules.activation.Tanh,
  torch.nn.modules.activation.Softmin,
  torch.nn.modules.activation.Tanh,
  torch.nn.modules.activation.Tanh,
  torch.nn.modules.activation.PReLU,
  torch.nn.modules.activation.LeakyReLU],
 'neuron_cnt': array([5, 5, 5, 5, 5, 5, 5, 5])}

In [19]:
ind3 = ind1.crossover(ind2)
vars(ind3.genome)

{'params': <genome_parameters.GenomeParameters at 0x28bd185c0>,
 'epochs': 21,
 'batch_size': 68,
 'hidden_layers': 8,
 'activation_fns': [torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.Softmax,
  torch.nn.modules.activation.Hardswish,
  torch.nn.modules.activation.GELU,
  torch.nn.modules.activation.PReLU,
  torch.nn.modules.activation.GELU,
  torch.nn.modules.linear.Linear],
 'neuron_cnt': array([5, 5, 5, 5, 5, 5, 5, 5])}

# Testing fitness

In [4]:
from individual import Individual
from neural_network import NeuralNetwork
from fitness import FitnessEvaluator
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [10]:
ind = Individual(EvolutionParameters(mutation_prob, crossover_prob), 
                            GenomeParameters(#activation_functions=[torch.nn.Linear, torch.nn.ReLU],
                                             #min_epochs=10, max_epochs=10,
                                             min_hidden_neurons=256, max_hidden_neurons=1024
                                             ),
                            device='mps')
vars(ind.genome)



TypeError: EvolutionParameters.__init__() missing 4 required positional arguments: 'training_data', 'test_data', 'loss_fn', and 'optimizer'

In [22]:
#This is very clunky to use; find some way to have optimization function passed to fitnessevaluator (we generate nn twice otherwise)
nn = NeuralNetwork(ind.genome, input_layer=(torch.nn.Linear, 28*28), output_layer=(torch.nn.Linear, 10))
nn


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (model): Sequential(
    (input): Linear(in_features=784, out_features=1008, bias=True)
    (ELU0): ELU(alpha=1.0)
    (Sigmoid0): Sigmoid()
    (LeakyReLU0): LeakyReLU(negative_slope=0.01)
    (PReLU0): PReLU(num_parameters=1)
    (SELU0): SELU()
    (ELU1): ELU(alpha=1.0)
    (Softmin0): Softmin(dim=None)
    (Hardswish0): Hardswish()
    (Softmax0): Softmax(dim=None)
    (Softmin1): Softmin(dim=None)
    (Sigmoid1): Sigmoid()
    (GELU0): GELU(approximate='none')
    (PReLU1): PReLU(num_parameters=1)
    (Sigmoid2): Sigmoid()
    (ReLU0): ReLU()
    (GELU1): GELU(approximate='none')
    (Sigmoid3): Sigmoid()
    (GELU2): GELU(approximate='none')
    (GELU3): GELU(approximate='none')
    (Softmin2): Softmin(dim=None)
    (GELU4): GELU(approximate='none')
    (ReLU1): ReLU()
    (Softmax1): Softmax(dim=None)
    (SELU1): SELU()
    (SELU2): SELU()
    (GELU5): GELU(approximate='none')
    (Sigmoid4): Sigmoid()
    (PReLU2)

In [23]:
ind.genome.batch_size = 64

In [24]:
ind.genome.epochs = 25

In [25]:
eval = FitnessEvaluator(ind, training_data, test_data, 
                        torch.nn.CrossEntropyLoss(), 
                        torch.optim.SGD(nn.model.to(ind.device).parameters(), lr=1e-3),
                        ind.device,
                        nn = nn)

In [26]:
acc, loss = eval.evaluate()

Epoch 1
-------------------------------
loss: 2.303464  [   64/60000]
loss: 2.298164  [ 6464/60000]
loss: 2.303468  [12864/60000]
loss: 2.302395  [19264/60000]
loss: 2.306775  [25664/60000]
loss: 2.303379  [32064/60000]
loss: 2.303770  [38464/60000]
loss: 2.300689  [44864/60000]
loss: 2.303067  [51264/60000]
loss: 2.305223  [57664/60000]
Test Error: 
 Accuracy: 10.1%, Avg loss: 2.302714 

Epoch 2
-------------------------------
loss: 2.302847  [   64/60000]
loss: 2.298209  [ 6464/60000]
loss: 2.303136  [12864/60000]
loss: 2.302260  [19264/60000]
loss: 2.306907  [25664/60000]
loss: 2.303572  [32064/60000]
loss: 2.302887  [38464/60000]
loss: 2.301523  [44864/60000]
loss: 2.303262  [51264/60000]


KeyboardInterrupt: 

In [5]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
evo_params = EvolutionParameters(mutation_prob, crossover_prob,
                                 training_data, test_data,
                                 torch.nn.CrossEntropyLoss(),
                                 lambda x: torch.optim.SGD(x, lr=1e-3),
                                 input_layer=(torch.nn.Linear, 28*28),
                                 output_layer=(torch.nn.Linear, 10))


In [11]:
ind = Individual(evo_params, 
                            GenomeParameters(#activation_functions=[torch.nn.Linear, torch.nn.ReLU],
                                             min_epochs=3, max_epochs=3,
                                             min_hidden_neurons=256, max_hidden_neurons=1024,
                                             min_batch_size=64, max_batch_size=64
                                             ),
                            device='mps')
vars(ind.genome)



Epoch 1
-------------------------------


/Users/rderi/Programming/Class/COSC461-S24/cosc461_project/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.293381  [   64/60000]
loss: 2.299838  [ 6464/60000]
loss: 2.309453  [12864/60000]
loss: 2.301697  [19264/60000]
loss: 2.307805  [25664/60000]
loss: 2.307318  [32064/60000]
loss: 2.296962  [38464/60000]
loss: 2.310831  [44864/60000]


KeyboardInterrupt: 

In [9]:
ind.fitness

(array([2.30282717, 2.30252329, 2.30227114]), array([0.1009, 0.1009, 0.1009]))

In [8]:
ind.measure_fitness()

Epoch 1
-------------------------------


/Users/rderi/Programming/Class/COSC461-S24/cosc461_project/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/rderi/Programming/Class/COSC461-S24/cosc461_project/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.307581  [   16/60000]
loss: 2.304007  [ 1616/60000]
loss: 2.302469  [ 3216/60000]
loss: 2.299960  [ 4816/60000]
loss: 2.308159  [ 6416/60000]
loss: 2.305856  [ 8016/60000]
loss: 2.312680  [ 9616/60000]
loss: 2.311811  [11216/60000]
loss: 2.299243  [12816/60000]
loss: 2.304780  [14416/60000]
loss: 2.308098  [16016/60000]
loss: 2.299286  [17616/60000]
loss: 2.306631  [19216/60000]
loss: 2.303165  [20816/60000]
loss: 2.300147  [22416/60000]
loss: 2.303380  [24016/60000]
loss: 2.291083  [25616/60000]
loss: 2.297090  [27216/60000]
loss: 2.296386  [28816/60000]
loss: 2.311637  [30416/60000]
loss: 2.298645  [32016/60000]
loss: 2.306671  [33616/60000]
loss: 2.297541  [35216/60000]
loss: 2.307045  [36816/60000]
loss: 2.298802  [38416/60000]


KeyboardInterrupt: 

In [ ]:
ind.fitness

In [ ]:
eval = FitnessEvaluator(ind, E, 
                        torch.nn.CrossEntropyLoss(), 
                        torch.optim.SGD(nn.model.to(ind.device).parameters(), lr=1e-3),
                        ind.device,
                        nn = nn)

# Testing generations

In [1]:
from individual import Individual
from generation import Generation
from evolution_parameters import EvolutionParameters
from genome_parameters import GenomeParameters
from neural_network import NeuralNetwork
from fitness import FitnessEvaluator
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
population_cnt = 4
mutation_prob = 0.3
crossover_prob = 0.1

In [3]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
generation = Generation(population_cnt, 
                 EvolutionParameters(mutation_prob, crossover_prob,
                                     training_data, test_data,
                                     torch.nn.CrossEntropyLoss(),
                                     lambda x: torch.optim.SGD(x, lr=1e-3),
                                     tournament_size=2,
                                     input_layer=(torch.nn.Linear, 28*28),
                                     output_layer=(torch.nn.Linear, 10)), 
                 GenomeParameters(max_hidden_layers=10, max_epochs=3))

Epoch 1
-------------------------------
loss: 2.316954  [   28/60000]


/Users/rderi/Programming/Class/COSC461-S24/cosc461_project/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/rderi/Programming/Class/COSC461-S24/cosc461_project/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.289083  [ 2828/60000]
loss: 2.294587  [ 5628/60000]
loss: 2.320787  [ 8428/60000]
loss: 2.292530  [11228/60000]
loss: 2.300985  [14028/60000]
loss: 2.279706  [16828/60000]
loss: 2.315766  [19628/60000]
loss: 2.316736  [22428/60000]
loss: 2.304193  [25228/60000]
loss: 2.315670  [28028/60000]
loss: 2.283728  [30828/60000]
loss: 2.328009  [33628/60000]
loss: 2.318639  [36428/60000]
loss: 2.298093  [39228/60000]
loss: 2.302464  [42028/60000]
loss: 2.325456  [44828/60000]
loss: 2.307033  [47628/60000]
loss: 2.305549  [50428/60000]
loss: 2.299368  [53228/60000]
loss: 2.313679  [56028/60000]
loss: 2.314768  [58828/60000]
Test Error: 
 Accuracy: 9.8%, Avg loss: 2.304891 

Epoch 1
-------------------------------
loss: 2.344736  [    7/60000]
loss: 2.225368  [  707/60000]
loss: 2.268105  [ 1407/60000]
loss: 2.353578  [ 2107/60000]
loss: 2.394616  [ 2807/60000]
loss: 2.217835  [ 3507/60000]
loss: 2.267605  [ 4207/60000]
loss: 2.181513  [ 4907/60000]
loss: 2.467712  [ 5607/60000]
loss: 2.2

In [10]:
generation.progress_generation()

In [11]:
len(generation.population)

4